In [40]:
%pip install nltk pandas spacy stanza


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def lemmatize_text(text):
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

# Aplikácia lemmatizácie
df['lemmatized_tokens'] = df['tokens_no_stopwords'].apply(lambda tokens: lemmatize_text(' '.join(tokens)))
print("\nUkážka lemmatizovaných tokenov:")
print(df[['tokens_no_stopwords', 'lemmatized_tokens']].head())

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import stanza
import re
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializácia Stanza pre češtinu (na lemmatizáciu a POS tagging)
stanza.download('cs')  # Stiahnutie českého modelu
nlp = stanza.Pipeline(lang='cs', processors='tokenize,lemma,pos')

# Stiahnutie NLTK dát
nltk.download('punkt')

# Načítanie CSV súboru
# Predpokladáme, že CSV má stĺpec 'text' s českými textovými dátami
df = pd.read_csv('recenzie.csv')  # Nahraď 'data.csv' správnym názvom súboru
print("Prvé riadky dát:")
print(df.head())

# Základné štatistiky
print("\nZákladné informácie o dátach:")
print(df.info())

2025-05-15 10:45:28 INFO: Downloaded file to /home/codespace/stanza_resources/resources.json
2025-05-15 10:45:28 INFO: Downloading default packages for language: cs (Czech) ...
2025-05-15 10:45:28 INFO: File exists: /home/codespace/stanza_resources/cs/default.zip
2025-05-15 10:45:30 INFO: Finished downloading models and saved to /home/codespace/stanza_resources
2025-05-15 10:45:30 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-05-15 10:45:30 INFO: Downloaded file to /home/codespace/stanza_resources/resources.json
2025-05-15 10:45:30 WARNING: Language cs package default expects mwt, which has been added
2025-05-15 10:45:32 INFO: Loading these models for language: cs (Czech):
| Processor | Package      |
----------------------------
| tokenize  | pdt          |
| mwt       | pdt          |
| pos       | pdt_nocharlm |
| lemma     | pdt_no

Prvé riadky dát:
   Hviezdičky             Meno                             Pocet_recenzii  \
0           1   Valerie Šálová                                  2 recenze   
1           4   Václav Namyslo   Místní průvodce · 99 recenzí · 150 fotek   
2           1              J P                                  1 recenze   
3           4       Jan Franek   Místní průvodce · 106 recenzí · 33 fotek   
4           5  Miroslav Vlasak  Místní průvodce · 241 recenzí · 135 fotek   

                                                Text       Datum  
0  Obsluha od pohledu velice nepříjemná, na výběr...  2025-05-12  
1  Ale jo. Jídlo z menu bylo dobré. Akorát špenát...  2025-05-15  
2  Čekali jsme velmi dlouho, obsluha nepříjemná. ...  2025-05-15  
3  Příjemná restaurace!\r\n\r\nDo restaurace jsem...  2024-09-17  
4  Měli jsme syrečkový tatarák a s mažené gorbáči...  2025-05-15  

Základné informácie o dátach:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (t

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
#Lowercase a čistenie textu

def clean_text(text):
    # Prevod na malé písmená
    text = text.lower()
    text = ' '.join(text.split())
    return text

# Aplikácia čistenia na stĺpec 'text'
df['cleaned_text'] = df['Text'].apply(clean_text)
print("\nUkážka vyčisteného textu:")
print(df[['Text', 'cleaned_text']].head())


Ukážka vyčisteného textu:
                                                Text  \
0  Obsluha od pohledu velice nepříjemná, na výběr...   
1  Ale jo. Jídlo z menu bylo dobré. Akorát špenát...   
2  Čekali jsme velmi dlouho, obsluha nepříjemná. ...   
3  Příjemná restaurace!\r\n\r\nDo restaurace jsem...   
4  Měli jsme syrečkový tatarák a s mažené gorbáči...   

                                        cleaned_text  
0  obsluha od pohledu velice nepříjemná, na výběr...  
1  ale jo. jídlo z menu bylo dobré. akorát špenát...  
2  čekali jsme velmi dlouho, obsluha nepříjemná. ...  
3  příjemná restaurace! do restaurace jsem přišel...  
4  měli jsme syrečkový tatarák a s mažené gorbáči...  


In [34]:
def tokenize_text(text):
    doc = nlp(text)
    return [word.text for sent in doc.sentences for word in sent.words]

# Aplikácia tokenizácie
df['tokens'] = df['cleaned_text'].apply(tokenize_text)
print("\nUkážka tokenizovaného textu:")
print(df[['cleaned_text', 'tokens']].head())


Ukážka tokenizovaného textu:
                                        cleaned_text  \
0  obsluha od pohledu velice nepříjemná, na výběr...   
1  ale jo. jídlo z menu bylo dobré. akorát špenát...   
2  čekali jsme velmi dlouho, obsluha nepříjemná. ...   
3  příjemná restaurace! do restaurace jsem přišel...   
4  měli jsme syrečkový tatarák a s mažené gorbáči...   

                                              tokens  
0  [obsluha, od, pohledu, velice, nepříjemná, ,, ...  
1  [ale, jo, ., jídlo, z, menu, bylo, dobré, ., a...  
2  [čekali, jsme, velmi, dlouho, ,, obsluha, nepř...  
3  [příjemná, restaurace, !, do, restaurace, jsem...  
4  [měli, jsme, syrečkový, tatarák, a, s, mažené,...  


In [38]:

# Načítanie stop slov z GitHubu
# používame české stop slová z repozitára stopwords-iso link
stopwords_url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-cs/master/stopwords-cs.json'
response = requests.get(stopwords_url)
czech_stop_words = set(json.loads(response.text))

def remove_stopwords(tokens):
    return [token for token in tokens if token not in czech_stop_words]

# Odstránenie stop slov
df['tokens_no_stopwords'] = df['tokens'].apply(remove_stopwords)
print("\nUkážka tokenov po odstránení stop slov:")
print(df[['tokens', 'tokens_no_stopwords']].head())


Ukážka tokenov po odstránení stop slov:
                                              tokens  \
0  [obsluha, od, pohledu, velice, nepříjemná, ,, ...   
1  [ale, jo, ., jídlo, z, menu, bylo, dobré, ., a...   
2  [čekali, jsme, velmi, dlouho, ,, obsluha, nepř...   
3  [příjemná, restaurace, !, do, restaurace, jsem...   
4  [měli, jsme, syrečkový, tatarák, a, s, mažené,...   

                                 tokens_no_stopwords  
0  [obsluha, pohledu, velice, nepříjemná, ,, výbě...  
1  [jo, ., jídlo, menu, dobré, ., akorát, špenát,...  
2  [čekali, velmi, dlouho, ,, obsluha, nepříjemná...  
3  [příjemná, restaurace, !, restaurace, přišel, ...  
4  [měli, syrečkový, tatarák, mažené, gorbáčiky, ...  


In [ ]:
#Ceska lematizácia
def lemmatize_text(text):
    doc = nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

# Aplikácia lemmatizácie
df['lemmatized_tokens'] = df['tokens_no_stopwords'].apply(lambda tokens: lemmatize_text(' '.join(tokens)))
print("\nUkážka lemmatizovaných tokenov:")
print(df[['tokens_no_stopwords', 'lemmatized_tokens']].head())


Ukážka lemmatizovaných tokenov:
                                 tokens_no_stopwords  \
0  [obsluha, pohledu, velice, nepříjemná, ,, výbě...   
1  [jo, ., jídlo, menu, dobré, ., akorát, špenát,...   
2  [čekali, velmi, dlouho, ,, obsluha, nepříjemná...   
3  [příjemná, restaurace, !, restaurace, přišel, ...   
4  [měli, syrečkový, tatarák, mažené, gorbáčiky, ...   

                                   lemmatized_tokens  
0  [obsluha, pohled, velice, příjemný, ,, výběr, ...  
1  [jo, ., jídlo, menu, dobrý, ., akorát, špenát,...  
2  [čekat, velmi, dlouho, ,, obsluha, příjemný, ....  
3  [příjemný, restaurace, !, restaurace, přijít, ...  
4  [mít, syrečkový, tatarák, mažený, gorbáčika, ,...  


In [ ]:
# Inicializácia TF-IDF vektorizéra
vectorizer = TfidfVectorizer(max_features=1000)  # Obmedzíme na 1000 najčastejších slov
tfidf_matrix = vectorizer.fit_transform(df['lemmatized_tokens'].apply(lambda x: ' '.join(x)))
print("\nUkážka TF-IDF matice:")

# Prevedenie na DataFrame pre lepšiu prehľadnosť
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print("\nUkážka TF-IDF matice:")
print(tfidf_df.head())


Ukážka TF-IDF matice:

Ukážka TF-IDF matice:
    00   10   11   12   13   14   15  170   18   20  ...  špinava  špinavej  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...      0.0       0.0   

   špinavý  štěkat  štěstí     šunka  žampion  žebro  židlička  žádný  
0      0.0     0.0     0.0  0.296158      0.0    0.0       0.0    0.0  
1      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  
2      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  
3      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  
4      0.0     0.0     0.0  0.000000      0.0    0.0       0.0    0.0  

[5 rows x 1000 columns]
